In [2]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd 
import time
import os, requests

In [26]:
def scrape(path):
    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(r"D:\GS_WebScarping\project_2\chromedriver.exe")
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)

# =====================================================================================================
    # complete login
# =====================================================================================================
    # provide username 
    try:
        provided_username = "haider1805@icloud.com"
        username_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "loginForm.Email")))
        # username_field = driver.find_element((By.ID, "loginForm.Email"))
        # send the username written above
        username_field.send_keys(provided_username)
    except Exception as e:
        print(f"No email button found and the error is--> {e}")
        
    # get the password id from website
    try:
        provided_password = "Muhssan7865"
        # pass_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "loginForm.Password")))
        pass_field = driver.find_element(By.ID, "loginForm.Password")
        pass_field.send_keys(provided_password)
    except Exception as e:
        print(f"No password tab found and the error is--> {e}")

    # get the login button name-value
    try:
        # login_button= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@value = 'Login']")))
        login_button = driver.find_element(By.XPATH, "//input[@value = 'Login']")
        login_button.click()
    except Exception as e:
        print(f"No login button found and error is --> {e}")
# =====================================================================================================
# =====================================================================================================   


# =====================================================================================================
    # get the number of cars present for display
# =====================================================================================================
    try:
        num_cars = driver.find_element(By.CSS_SELECTOR, ".ed__value_mb")
        num_cars=int(num_cars.text)
        print(f"Total {num_cars} number of cars found")
    except Exception as e:
        print(f"No number of car tab found and the error is --> {e}")


# =====================================================================================================
    # get car page links and click the very first link
# =====================================================================================================
    try:
        links = driver.find_element(By.CSS_SELECTOR, "a.car__title")
        if links:
            links.click()
        else:
            print("No car link found")
    except Exception as e:
        print(f"No car link found and the error is --> {e}")



# =====================================================================================================
    # loop around the number of cars
# =====================================================================================================
    # list to append the values
    results = []
    for i in range(num_cars): # len(num_cars)
        # dictionary 
        details = {}
        
        # get the whole card
        try:
            car_complete_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@id="ActiveVehicleDetail"]')))
            if car_complete_card:
 
                # get the name of the car
                try:
                    car_tilte = WebDriverWait(car_complete_card, 10).until(EC.presence_of_element_located((By.XPATH, './/h1[@class="vehicle-details__title"]'))) 
                    # driver.find_element(By.CLASS_NAME, "vehicle-details__title")
                    if car_tilte:
                        details['Name'] = car_tilte.text.strip()
                    else:
                        details['Name'] = "No car name found"
                except Exception as e:
                    print(f"No title ")
                    
                    
                # =====================================================================================================
                # get the gearbox, fuel, cc and vehicle type 
                # =====================================================================================================
                try:
                    car_properties = WebDriverWait(car_complete_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-details__properties "]')))  
                    if car_properties:
                        details["Car_properties"] = car_properties.text.strip()
                    else:
                        print("No car properties found")
                except Exception as e:
                    print(f"No car properties tab found")
                    
                
                # get the image overview bar --> done
                try:
                    img_main_card =WebDriverWait(car_complete_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="flexslider flexslider__vehicle js-vehicle-image"]')))  
                    if img_main_card:
                        imgs_ul =WebDriverWait(img_main_card, 10).until(EC.presence_of_element_located((By.XPATH, './/ul[@class="slides"]')))
                        if imgs_ul:
                            images = []
                            imgs_lis = WebDriverWait(imgs_ul, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "li")))
                            if imgs_lis:
                                for img in imgs_lis:
                                    images.append(img.get_attribute("data-thumb"))

                            img_str = ", ".join(images)
                            details["Images"] =  img_str
                except Exception as e:
                    print("No images found")
                    
                
                # =====================================================================================================
                # get the veh information, provenance and sales and event info
                # =====================================================================================================      
                # vehicle information -->done
                try:       
                    veh_main_card= WebDriverWait(car_complete_card, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "vehicle-details__top-content")))  
                    if veh_main_card:
                        
                        # veh details ---> done
                        try:
                            veh_info_card = veh_main_card.find_element(By.XPATH, ".//div[@class = 'vehicle-details__block vdb']")
                            veh_cont = veh_info_card.find_element(By.XPATH, "//div[@class = 'vdb__content js-dropdown-container']")
                            if veh_cont:
                                # get the car_info_items
                                car_info_items = veh_cont.find_elements(By.CLASS_NAME, "vdb__item")
                                # inside each info_item
                                for info_item in car_info_items:
                                    # get label
                                    info_label = info_item.find_element(By.CLASS_NAME, "vdb__label").text
                                    info_value = info_item.find_element(By.CLASS_NAME, "vdb__value").text
                                    if info_label and info_value:
                                        # get value
                                        details[info_label] = info_value
                                    else:
                                        details[info_label] = "N/A"
                            else:
                                print("No veh info card found")
                        except Exception as e:
                            print("No veh details")
                            
                            
                        # prov data ---> done
                        try:
                            prov_id_card = WebDriverWait(veh_main_card, 10).until(EC.presence_of_element_located((By.XPATH, ".//div[@id='divProvenenceCheckSummary']")))
                            prov_data_card = prov_id_card.find_element(By.XPATH, ".//div[@class = 'vdb__content js-dropdown-container']")
                            if prov_data_card:
                                
                                prov_items = prov_data_card.find_elements(By.CLASS_NAME, "vdb__item")
                                for prov_item in prov_items:
                                    # get label info
                                    prov_label = prov_item.find_element(By.CLASS_NAME, "vdb__label").text
                                    prov_value = prov_item.find_element(By.CLASS_NAME, "vdb__value").text
                                    if prov_label and prov_value:
                                        details[prov_label] = prov_value
                                    else:
                                        details[prov_label] = "N/A"
                            else:
                                details[prov_label] = "N/A"
                                                        
                        except Exception as e:
                            print(f"No prov value found")
                            
                        # sales ----> done                        
                        try:
                            sales_card = WebDriverWait(veh_main_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-details__block vdb vdb_sales"]')))
                            if sales_card:
                                sales_data = WebDriverWait(sales_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__content js-dropdown-container"]')))
                                if sales_data:
                                    sales_items = sales_data.find_elements(By.XPATH, './/div[@class= "vdb__item"]')
                                    for sales_item in sales_items:
                                        try:
                                            # Extract label
                                            sales_label = sales_item.find_element(By.XPATH, './/div[@class= "vdb__label"]').text.strip()

                                            # Extract value
                                            sales_value = ""
                                            value_element = sales_item.find_element(By.XPATH, './/span[@class= "vdb__value"]')
                                            if value_element:
                                                # Check if value is inside an <a> tag
                                                link = value_element.find_elements(By.XPATH, './/a')
                                                if link:
                                                    sales_value = link[0].text.strip()
                                                else:
                                                    sales_value = value_element.text.strip()

                                            # Store in the details dictionary
                                            details[sales_label] = sales_value
                                        except Exception as e:
                                            print(f"Error extracting sales item")
                                else:
                                    print("No data items for sales card")
                            else:
                                print("No sales card")
                        except Exception as e:
                            pass         
                except Exception as e:
                        print(f"No veh information values")
                        
            
                
                # get the event name date/time and lot number --> done
                try:
                    main_right_card = WebDriverWait(car_complete_card, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.sidebar_right")))
                    if main_right_card:
                        # acces the event name
                        event_name = main_right_card.find_element(By.CSS_SELECTOR, "span.widget-event-name")
                        if event_name:
                            details["Event name"] = event_name.text
                        else:
                            details["Event name"] = "N/A"

                        # acces the date
                        date_time = main_right_card.find_element(By.ID, "startDate")
                        if date_time:
                            details['Date_time'] = date_time.text
                        else:
                            details['Date_time'] = "N/A"
                        
                        # acces Lot Number
                        lot_loc = main_right_card.find_elements(By.CSS_SELECTOR, ".block__row")
                        if lot_loc:
                            for i in lot_loc:
                                label = i.find_element(By.CSS_SELECTOR, '.block__column').text
                                if label:
                                    value = i.find_element(By.CSS_SELECTOR, "div.block__column.bold").text
                                    if value:
                                        details[label] = value
                                    else:
                                        details[label] = "N/A"
                                else:
                                    print("No label in lot and loc ")
                    else:
                        print("No right bar availabel")
                except Exception as e:
                    print(f"No right bar found")
                
                
                # get the specifications and adds --> done
                try:
                    prov_specs_adds_cards = WebDriverWait(car_complete_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="vehicle-details__block vehicle-details__clear vdb"]')))
                    spec_card = prov_specs_adds_cards[0]
                            
                    # specs
                    try:
                        specs_data_card = WebDriverWait(spec_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__content js-dropdown-container"]'))) 
                        specs_card_data_2 = WebDriverWait(specs_data_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__content_text"]')))
                        specs = WebDriverWait(specs_card_data_2, 10).until(EC.presence_of_element_located((By.XPATH, './/ul[@class="vdb__columns"]')))
                        if specs:
                            specs_dets = specs.find_elements(By.TAG_NAME, 'li')
                            specs_info = []
                            for spec_el in specs_dets:
                                specs_info.append(spec_el.text.strip())
                                
                            details['Specification'] = specs_info
                        else:
                            details['Specification'] = 'na'
                    except Exception as e:
                        print("No specs card found")
                        
                        
                    # adds 
                    adds_card =prov_specs_adds_cards[1]
                    try:
                        adds_data_card = WebDriverWait(adds_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__content js-dropdown-container"]')))
                        adds_data = WebDriverWait(adds_data_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__content_text"]')))
                        # adds_card.find_element(By.XPATH, './/div[@class="vdb__content_text"]/following::ul')
                        if adds_data:
                            adds_dets = adds_data.find_element(By.TAG_NAME, 'li')
                            details['Add: Information'] = adds_dets.text.strip()
                        else:
                            details['Add: Information'] = 'na'
                    except Exception as e:
                        print("No specs card found")                    

                except Exception as e:
                    print("no specs and adds cards found")
                
   
                # get the pricing information --> done
                try:
                    pricing_card=WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-details__block vdb vdb_grey vdb_valuations"]')))
                    if pricing_card:
                        pricing_data = WebDriverWait(pricing_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__content vdb__content_table js-dropdown-container"]')))
                        if pricing_data:
                            pricing_items = WebDriverWait(pricing_data, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="vdb__item"]')))
                            if pricing_items:
                                for pricing_item in pricing_items:
                                        pricing_labels = pricing_item.find_element(By.CLASS_NAME, "vdb__label").text.strip()
                                        pricing_value = pricing_item.find_element(By.CLASS_NAME, "vdb__value")
                                        if pricing_labels and pricing_value:
                                            details[pricing_labels] = pricing_value.text.strip()
                                        else:
                                            details[pricing_labels] = "N/A"
                            else:
                                print("No pricing labels found")
                        else:
                            print("No pricing table found")
                    else:
                        print("No pricing items found")
                except Exception as e:
                    print(f"No pricing tab found")
                
                
                # inspection --> done
                try:
                    # Wait for all blocks with the class `vehicle-details___block vdb`
                    vehicle_details_blocks = WebDriverWait(car_complete_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="vehicle-details__block vdb"]')))
                    print(len(vehicle_details_blocks))
                    
                    for block in vehicle_details_blocks:
                        try:
                            # Check if the block title matches "Inspection & Service History"
                            block_titles = block.find_elements(By.XPATH, './/a[@class="vdb__title js-dropdown-onmobile"]')
                            for block_title in block_titles:
                                if block_title.text.strip() == "Inspection & Service History":
                                    print(f"Found the block: {block_title.text.strip()}")
                                    
                                    # Find all items within this block
                                    details_items = block.find_elements(By.XPATH, './/div[@class="vdb__item"]')
                                    
                                    if details_items:
                                        for item in details_items:
                                            # Extract label and value for each item
                                            label = item.find_element(By.XPATH, './/div[@class="vdb__label"]').text.strip()
                                            value_element = item.find_element(By.XPATH, './/span[@class="vdb__value"]')
                                            
                                            # Handle cases where value contains links
                                            if value_element.find_elements(By.TAG_NAME, 'a'):
                                                value = value_element.find_element(By.TAG_NAME, 'a').get_attribute("href")
                                            else:
                                                value = value_element.text.strip()
                                            
                                            # Add the label and value to the details dictionary
                                            details[label] = value
                                    else:
                                        print("No details items found in the block")
                                    break
                                
                                # Stop further processing once the desired block is found
                                break
                        
                        except Exception as e:
                            print(f"Error processing a block")

                except Exception as e:
                    print(f"Error locating vehicle details blocks")
                
                    
                
                # detailed hist --> done
                try:
                    det_hist_card = WebDriverWait(car_complete_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-details__block vdb vdb_table"]')))
                    if det_hist_card:
                        det_table =WebDriverWait(det_hist_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__table"]'))) 
                        headers = WebDriverWait(det_table, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__row vdb__row_header"]')))
                        if headers:
                            cols = headers.find_elements(By.XPATH, './/div[@class="vdb__td vdb__th"]')    
                            cols_list = [col.text.strip() for col in cols] 
                            
                        else:
                            print('no columns of detailed info')
                        
                        rows =[]
                        vals = WebDriverWait(det_table, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="vdb__row"]')))      
                        if vals:
                                for val in vals:
                                    data_elements = val.find_elements(By.CLASS_NAME, "vdb__td")
                                    row_data = [data.text.strip() for data in data_elements]
                                    if row_data:  # Avoid empty rows
                                        rows.append(row_data)
                        else:
                            print("No detailed hist values")
                        
                        # detailed_list = 
                        details['Detailed_hist'] = cols_list + rows
                except Exception as e:
                    print(f"no detailed info")   
                
                
                
                # Mot hist --> done
                try:
                    mot_hist_card = WebDriverWait(car_complete_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@id="divProvenenceCheckDetail"]')))
                    if mot_hist_card:
                        mot_table =WebDriverWait(mot_hist_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__table"]'))) 
                        # columns
                        mot_headers = WebDriverWait(mot_table, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__row vdb__row_header"]')))
                        if mot_headers:
                            mot_cols = mot_headers.find_elements(By.XPATH, './/div[@class="vdb__td vdb__th"]')    
                            mot_cols_list = [mot_col.text.strip() for mot_col in mot_cols] 
                            # print(mot_cols_list)
                            
                        else:
                            print('no columns of detailed info')
                        
                        mot_rows =[]
                        mot_vals = WebDriverWait(mot_table, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="vdb__row"]')))      
                        if mot_vals:
                                for mot_val in mot_vals:
                                    mot_data_elements = mot_val.find_elements(By.CLASS_NAME, "vdb__td")
                                    mot_row_data = [mot_data.text.strip() for mot_data in mot_data_elements]
                                    if row_data:  # Avoid empty rows
                                        mot_rows.append(mot_row_data)
                        else:
                            print("No mot hist values")
                        
                        # detailed_list = 
                        details['MOT_hist'] = mot_cols_list + mot_rows
                except Exception as e:
                    print(f"no mot info")   
                
                
                results.append(details)


                # =====================================================================================
                    # get the next button
                # =====================================================================================
                try:
                    next_main_card = driver.find_element(By.CSS_SELECTOR, ".vehicle-navigation__prev_next_div")
                    next_button = next_main_card.find_element(By.XPATH, "//a[text()='Next ']")
                    if next_button:
                        #     
                        next_button.click()
                    else:
                        print("No next button")
                        break
                except Exception as e:
                    print("No next button found hence no more cars are available")   
            else:
                print("No complete car card found")
        except Exception as e:
            print(f"No complete car card found and error is {e}")
    

    df = pd.DataFrame.from_dict(results)
    df.to_csv("manheim.csv", index=False)
    driver.quit()
path = "https://www.manheim.co.uk/catalogues-and-events/listing/43435?locationId=15&locationName=Leeds"
scrape(path)

Total 5 number of cars found
No prov value found
Error extracting sales item
No specs card found
5
Found the block: Inspection & Service History
no mot info
5
Found the block: Inspection & Service History
4
Found the block: Inspection & Service History
no mot info
No prov value found
Error extracting sales item
No specs card found
4
Found the block: Inspection & Service History
no mot info
5
Found the block: Inspection & Service History
no mot info
No next button found hence no more cars are available


In [29]:
df =pd.read_csv("manheim.csv")
df.columns

Index(['Name', 'Car_properties', 'Images', 'REG NO', 'MFR YEAR', 'REG DATE',
       'ODOMETER', 'BASE COLOUR', 'COLOUR', 'COLOUR TYPE', 'INTERIOR COLOUR',
       'TRIM TYPE', 'NO OF SEATS', 'NO OF KEYS', 'V5', 'VAT', 'TOTAL LOSS',
       'RUNNER', 'IMPORTED', 'AUCTION CENTRE', 'VEHICLE LOCATION', 'VENDOR',
       'Event name', 'Date_time', 'Lot number', 'Event Location',
       'Add: Information', 'CAP CLEAN', 'CAP AVERAGE', 'CAP RETAIL', 'GRADE',
       'INSPECTION REPORT', 'REGISTERED KEEPERS', 'NO OF SERVICES',
       'DATE OF LAST SERVICE', 'ODOMETER AT LAST SERVICE', 'MOT EXPIRY DATE',
       'Detailed_hist', 'PRIVATE HIRE', 'MOT MILEAGE DISCREPANCY',
       'Specification', 'MOT_hist'],
      dtype='object')

### issue in sales, detailed hist and mot cards

In [30]:
# separate the reg no and images
reg_img = df[["REG NO", "Images"]]
reg_img

,REG NO,Images
0,PK13EBX,http://images.manheim.co.uk/images/units/ims_t...
1,PN16MVU,http://images.manheim.co.uk/images/units/ims_t...
2,SW14TYT,http://images.manheim.co.uk/images/units/ims_t...
3,MH05AOR,http://images.manheim.co.uk/images/units/ims_t...
4,FY67DGZ,http://images.manheim.co.uk/images/units/ims_t...


In [31]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["REG NO"] # separate reg nums
        image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)


Downloaded: Images\PK13EBX\PK13EBX_1_3709358-9999999-24114227-004.jpg
Downloaded: Images\PK13EBX\PK13EBX_2_3709358-9999999-24114227-008.jpg
Downloaded: Images\PK13EBX\PK13EBX_3_3709358-9999999-24114227-001.jpg
Downloaded: Images\PK13EBX\PK13EBX_4_3709358-9999999-24114227-002.jpg
Downloaded: Images\PK13EBX\PK13EBX_5_3709358-9999999-24114227-003.jpg
Downloaded: Images\PK13EBX\PK13EBX_6_3709358-9999999-24114227-005.jpg
Downloaded: Images\PK13EBX\PK13EBX_7_3709358-9999999-24114227-006.jpg
Downloaded: Images\PK13EBX\PK13EBX_8_3709358-9999999-24114227-007.jpg
Downloaded: Images\PK13EBX\PK13EBX_9_3709358-9999999-24114227-009.jpg
Downloaded: Images\PK13EBX\PK13EBX_10_3709358-9999999-24114227-010.jpg
Downloaded: Images\PK13EBX\PK13EBX_11_3709358-9999999-24114227-011.jpg
Downloaded: Images\PK13EBX\PK13EBX_12_3709358-9999999-24114227-012.jpg
Downloaded: Images\PK13EBX\PK13EBX_13_insp-0001-3709358-3654330-13.jpg
Downloaded: Images\PK13EBX\PK13EBX_14_insp-0001-3709358-3654330-1000.jpg
Downloaded: I